In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import ipywidgets as widgets
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
players = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/players.csv')
seasons = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/seasons.csv')
awards = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/awards.csv')
teams = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/teams.csv')
train = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/train.csv')


In [ ]:
train.head()

In [ ]:
def The_unpack_json_func(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)


In [ ]:
print(f"Dataset Shape: ",train.shape)

In [ ]:
train.info()

*  date:No null values with incorrect data type it should be datetime. 
*  nextDayPlayerEngagement:nested json format 
*  games:nested json format.
*  rosters:nested json format. 
*  playerBoxScores:  nested json format.
*  teamBoxScores: nested json format. 
*  transactions:nested json format.  
*   standings:nested json format.
*  awards:nested json format****.
*  events:nested json format.
*  playerTwitterFollowers:nested json format.
* teamTwitterFollowers:nested json format

here we notice that the datatype for date should be date and not int64.

In [ ]:
train['date']=pd.to_datetime(train['date'],format="%Y%m%d")
train.head()
type(train['date'])

Before moving I want to know what I want o predict from the data .So for that I will first go through sample submission file.

In [ ]:
example_sample_submission = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv")
example_sample_submission

going through this info,I think that in this project we need to predict the future fan engagement(this is also written in the data info available at kaggle) such as "reaction" and "behaviour" with the digital content of each NLB player ID . We will predict four different measures of engagement (target1-target4), each quantified on a scale of 0-100 with a different index.

 Guess from what information? ( looking  through the test data first)

In [ ]:
example_test = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/example_test.csv")
example_test

In [ ]:
example_test.head(3)

Visualize the example_test["games"].iloc[0]

In [ ]:
The_unpack_json_func(example_test["games"].iloc[0])

Visualise for roasters.iloc[0]

In [ ]:
The_unpack_json_func(example_test["rosters"].iloc[0])

From the info in this area ,I think its the estimate about the competition regarding the estimation of the expected values of the targets1-4 on the next day for each player.

Coming back to training data

the contents of train.csv are also in json file format in one cell, and it has a complicated shape like dataframe is further contained.So lets uncomplicate it.

In [ ]:
def opencol(col,n):
    tmp = train[col]
    tmp = tmp.dropna()
    tmpdf = The_unpack_json_func(tmp.iloc[n])
    print(tmpdf.columns)
    return tmpdf

In [ ]:
train_next_day = opencol("nextDayPlayerEngagement",0)
train_next_day

In [ ]:
train_next_day.head()

In [ ]:
unique_date = list(train_next_day.engagementMetricsDate.unique())
target1_lis = []
target2_lis = []
target3_lis = []
target4_lis = []
for i in unique_date:
    df = train_next_day[train_next_day['engagementMetricsDate']==i]
    target1 = (df['target1'].sum())/len(df)
    target2 = (df['target2'].sum())/len(df)
    target3 = (df['target3'].sum())/len(df)
    target4 = (df['target4'].sum())/len(df)
    target1_lis.append(target1)
    target2_lis.append(target2)
    target3_lis.append(target3)
    target4_lis.append(target4)
target1_lis
target2_lis
   

In [ ]:

px.line(df,x=unique_date,y=target1_lis,title='target 1 over time')


In [ ]:
import plotly.express as px
px.line(x=unique_date,y=target2_lis,title='target 2 over time')

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
fig = px.line(x=unique_date, y=target2_lis)
iplot(fig)


In [ ]:
import plotly.express as px
px.line(df,x=unique_date,y=target4_lis,title='target 4 over time')


In [ ]:
team_twitter = opencol("teamTwitterFollowers",0)
team_twitter

In [ ]:
standings = opencol("standings",0)
transactions = opencol("transactions",0)
standings

In [ ]:
transactions = transactions.dropna()

In [ ]:
most_transaction_team=transactions.groupby('fromTeamName')['toTeamName'].count().reset_index(name='Count').sort_values('Count',ascending=False)
px.bar(most_transaction_team.head(10),x='fromTeamName',y='Count')


In [ ]:
most_transaction_player=transactions.groupby('playerName')['toTeamName'].count().reset_index(name='Count').sort_values('Count',ascending=False)
px.bar(most_transaction_player.head(10),x='playerName',y='Count')

In [ ]:
players.head()

In [ ]:
players.info()

In [ ]:
px.histogram(players,x='birthCountry',color='birthCountry')

In [ ]:
data_dir = "../input/mlb-player-digital-engagement-forecasting"
df_names=['seasons','teams','players','awards']
kaggle_data_tabs = widgets.Tab()
# Display each DataFrame in widgets as a tab inside children
kaggle_data_tabs.children = list([widgets.Output() for df_name in df_names])


In [ ]:
for i in range(len(df_names)):
    kaggle_data_tabs.set_title(i, df_names[i])
    
    df = pd.read_csv(os.path.join(data_dir,df_names[i]) + ".csv")
    
    # Embed DataFrame in each tab
    with kaggle_data_tabs.children[i]:
        display(df)

In [ ]:
kaggle_data_tabs

In [ ]:
seasons = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/seasons.csv")
seasons
seasons['seasonStartDate']=pd.to_datetime(seasons['seasonStartDate'])
seasons['seasonEndDate']=pd.to_datetime(seasons['seasonEndDate'])
seasons['preSeasonStartDate']=pd.to_datetime(seasons['preSeasonStartDate'])
seasons['preSeasonEndDate']=pd.to_datetime(seasons['preSeasonEndDate'])
seasons['regularSeasonStartDate']=pd.to_datetime(seasons['regularSeasonStartDate'])
seasons['regularSeasonEndDate']=pd.to_datetime(seasons['regularSeasonEndDate'])
seasons['lastDate1stHalf']=pd.to_datetime(seasons['lastDate1stHalf'])
seasons['allStarDate']=pd.to_datetime(seasons['allStarDate'])
seasons['firstDate2ndHalf']=pd.to_datetime(seasons['firstDate2ndHalf'])
seasons['postSeasonStartDate']=pd.to_datetime(seasons['postSeasonStartDate'])
seasons['postSeasonEndDate']=pd.to_datetime(seasons['seasonStartDate'])
seasons


*  seasonId
*  seasonStartDate
* seasonEndDate
* preSeasonStartDate
* preSeasonEndDate
* regularSeasonStartDate
* regularSeasonEndDate
* lastDate1stHalf
* allStarDate
* firstDate2ndHalf
* postSeasonStartDate
* postSeasonEndDate


In [ ]:
players = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/players.csv")
players

In [ ]:
awards = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/seasons.csv")
awards

In [ ]:
for name in df_names:
    globals()[name] = pd.read_csv(os.path.join(data_dir,name)+ ".csv")

In [ ]:
daily_data_unnested_dfs = pd.DataFrame(data = {
  'dfName': train.drop('date', axis = 1).columns.values.tolist()
  })
daily_data_unnested_dfs['df'] = [pd.DataFrame() for row in 
  daily_data_unnested_dfs.iterrows()]
for df_index,df_row in daily_data_unnested_dfs.iterrows():
    nesteddatatableName= str(df_row['dfName'])
    date_nested_table=train[['date',nesteddatatableName]]
    #Removing the null values
    date_nested_table=(date_nested_table[~pd.isna(date_nested_table[nesteddatatableName])].reset_index(drop=True))
    daily_dfs_list=[]
    for date_index,date_row in date_nested_table.iterrows():
        daily_df= The_unpack_json_func(date_row[nesteddatatableName])
        daily_df['dailyDataDate'] = date_row['date']
        daily_dfs_list=daily_dfs_list+[daily_df]
    
    unnested_table = pd.concat(daily_dfs_list,ignore_index = True).set_index('dailyDataDate').reset_index()
        
# Creates 1 pandas df per unnested df from daily data read in, with same name
    globals()[df_row['dfName']] = unnested_table    
    
    daily_data_unnested_dfs['df'][df_index] = unnested_table

del train
gc.collect
  

In [ ]:
daily_data_unnested_dfs

In [ ]:
#### Get some information on each date in daily data (using season dates of interest) ####
dates = pd.DataFrame(data = 
  {'dailyDataDate': nextDayPlayerEngagement['dailyDataDate'].unique()})

dates['date'] = pd.to_datetime(dates['dailyDataDate'].astype(str))

dates['year'] = dates['date'].dt.year
dates['month'] = dates['date'].dt.month

dates_with_info = pd.merge(
  dates,
  seasons,
  left_on = 'year',
  right_on = 'seasonId'
  )
dates_with_info['inSeason'] = (
  dates_with_info['date'].between(
    dates_with_info['regularSeasonStartDate'],
    dates_with_info['postSeasonEndDate'],
    inclusive = True
    )
  )
dates_with_info['seasonPart'] = np.select(
  [
    dates_with_info['date'] < dates_with_info['preSeasonStartDate'], 
    dates_with_info['date'] < dates_with_info['regularSeasonStartDate'],
    dates_with_info['date'] <= dates_with_info['lastDate1stHalf'],
    dates_with_info['date'] < dates_with_info['firstDate2ndHalf'],
    dates_with_info['date'] <= dates_with_info['regularSeasonEndDate'],
    dates_with_info['date'] < dates_with_info['postSeasonStartDate'],
    dates_with_info['date'] <= dates_with_info['postSeasonEndDate'],
    dates_with_info['date'] > dates_with_info['postSeasonEndDate']
  ], 
  [
    'Offseason',
    'Preseason',
    'Reg Season 1st Half',
    'All-Star Break',
    'Reg Season 2nd Half',
    'Between Reg and Postseason',
    'Postseason',
    'Offseason'
  ], 
  default = np.nan
  )
#### Add some pitching stats/pieces of info to player game level stats ####

player_game_stats = (playerBoxScores.copy().
  # Change team Id/name to reflect these come from player game, not roster
  rename(columns = {'teamId': 'gameTeamId', 'teamName': 'gameTeamName'})
  )

# Adds in field for innings pitched as fraction (better for aggregation)
player_game_stats['inningsPitchedAsFrac'] = np.where(
  pd.isna(player_game_stats['inningsPitched']),
  np.nan,
  np.floor(player_game_stats['inningsPitched']) +
    (player_game_stats['inningsPitched'] -
      np.floor(player_game_stats['inningsPitched'])) * 10/3
  )

# Add in Tom Tango pitching game score (https://www.mlb.com/glossary/advanced-stats/game-score)
player_game_stats['pitchingGameScore'] = (40
#     + 2 * player_game_stats['outs']
    + 1 * player_game_stats['strikeOutsPitching']
    - 2 * player_game_stats['baseOnBallsPitching']
    - 2 * player_game_stats['hitsPitching']
    - 3 * player_game_stats['runsPitching']
    - 6 * player_game_stats['homeRunsPitching']
    )
# Add in criteria for no-hitter by pitcher (individual, not multiple pitchers)
player_game_stats['noHitter'] = np.where(
  (player_game_stats['gamesStartedPitching'] == 1) &
  (player_game_stats['inningsPitched'] >= 9) &
  (player_game_stats['hitsPitching'] == 0),
  1, 0
  )

player_date_stats_agg = pd.merge(
  (player_game_stats.
    groupby(['dailyDataDate', 'playerId'], as_index = False).
    # Some aggregations that are not simple sums
    agg(
      numGames = ('gamePk', 'nunique'),
      # Should be 1 team per player per day, but adding here for 1 exception:
      # playerId 518617 (Jake Diekman) had 2 games for different teams marked
      # as played on 5/19/19, due to resumption of game after he was traded
      numTeams = ('gameTeamId', 'nunique'),
      # Should be only 1 team for almost all player-dates, taking min to simplify
      gameTeamId = ('gameTeamId', 'min')
      )
    ),
  # Merge with a bunch of player stats that can be summed at date/player level
  (player_game_stats.
    groupby(['dailyDataDate', 'playerId'], as_index = False)
    [['runsScored', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'hits',
      'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
      'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
      'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
      'gamesStartedPitching', 'runsPitching', 'homeRunsPitching', 
      'strikeOutsPitching', 'baseOnBallsPitching', 'hitsPitching',
      'inningsPitchedAsFrac', 'earnedRuns', 
      'battersFaced','saves', 'blownSaves', 'pitchingGameScore', 
      'noHitter'
      ]].
    sum()
    ),
    on = ['dailyDataDate', 'playerId'],
  how = 'inner'
  )
#### Turn games table into 1 row per team-game, then merge with team box scores ####
# Filter to regular or Postseason games w/ valid scores for this part
games_for_stats = games[
  np.isin(games['gameType'], ['R', 'F', 'D', 'L', 'W', 'C', 'P']) &
  ~pd.isna(games['homeScore']) &
  ~pd.isna(games['awayScore'])
  ]
# Get games table from home team perspective
games_home_perspective = games_for_stats.copy()
# Change column names so that "team" is "home", "opp" is "away"
games_home_perspective.columns = [
  col_value.replace('home', 'team').replace('away', 'opp') for 
    col_value in games_home_perspective.columns.values]

games_home_perspective['isHomeTeam'] = 1

# Get games table from away team perspective
games_away_perspective = games_for_stats.copy()
# Put together games from home/away perspective to get df w/ 1 row per team game
team_games = (pd.concat([
  games_home_perspective,
  games_away_perspective
  ],
  ignore_index = True)
  )

# Copy over team box scores data to modify
team_game_stats = teamBoxScores.copy()

# Change column names to reflect these are all "team" stats - helps 
# to differentiate from individual player stats if/when joining later
team_game_stats.columns = [
  (col_value + 'Team') 
  if (col_value not in ['dailyDataDate', 'home', 'teamId', 'gamePk',
    'gameDate', 'gameTimeUTC'])
    else col_value
  for col_value in team_game_stats.columns.values
  ]



  
              

In [ ]:
 #Merge games table with team game stats
team_games_with_stats = pd.merge(
  team_games,
  team_game_stats.
    # Drop some fields that are already present in team_games table
    drop(['home', 'gameDate', 'gameTimeUTC'], axis = 1),
  on = ['dailyDataDate', 'gamePk', 'teamId'],
  # Doing this as 'inner' join excludes spring training games, postponed games,
  # etc. from original games table, but this may be fine for purposes here 
  how = 'inner'
  )
team_date_stats_agg = (team_games_with_stats.
  groupby(['dailyDataDate', 'teamId', 'gameType', 'oppId', 'oppName'], 
    as_index = False).
  agg(
    numGamesTeam = ('gamePk', 'nunique'),
    winsTeam = ('teamWinner', 'sum'),
    lossesTeam = ('oppWinner', 'sum'),
    runsScoredTeam = ('teamScore', 'sum'),
    runsAllowedTeam = ('oppScore', 'sum')
    )
   )
# Prepare standings table for merge w/ player digital engagement data
# Pick only certain fields of interest from standings for merge
standings_selected_fields = (standings[['dailyDataDate', 'teamId', 
  'streakCode', 'divisionRank', 'leagueRank', 'wildCardRank', 'pct'
  ]].
  rename(columns = {'pct': 'winPct'})
  )

# Change column names to reflect these are all "team" standings - helps 
# to differentiate from player-related fields if/when joining later
standings_selected_fields.columns = [
  (col_value + 'Team') 
  if (col_value not in ['dailyDataDate', 'teamId'])
    else col_value
  for col_value in standings_selected_fields.columns.values
  ]

standings_selected_fields['streakLengthTeam'] = (
  standings_selected_fields['streakCodeTeam'].
    str.replace('W', '').
    str.replace('L', '').
    astype(float)
    )
# Add fields to separate winning and losing streak from streak code
standings_selected_fields['winStreakTeam'] = np.where(
  standings_selected_fields['streakCodeTeam'].str[0] == 'W',
  standings_selected_fields['streakLengthTeam'],
  np.nan
  )

standings_selected_fields['lossStreakTeam'] = np.where(
  standings_selected_fields['streakCodeTeam'].str[0] == 'L',
  standings_selected_fields['streakLengthTeam'],
  np.nan
  )
standings_for_digital_engagement_merge = (pd.merge(
  standings_selected_fields,
  dates_with_info[['dailyDataDate', 'inSeason']],
  on = ['dailyDataDate'],
  how = 'left'
  ).
  # Limit down standings to only in season version
  query("inSeason").
  # Drop fields no longer necessary (in derived values, etc.)
  drop(['streakCodeTeam', 'streakLengthTeam', 'inSeason'], axis = 1).
  reset_index(drop = True)
  )
#### Merge together various data frames to add date, player, roster, and team info ####
# Copy over player engagement df to add various pieces to it
player_engagement_with_info = nextDayPlayerEngagement.copy()

# Take "row mean" across targets to add (helps with studying all 4 targets at once)
player_engagement_with_info['targetAvg'] = np.mean(
  player_engagement_with_info[['target1', 'target2', 'target3', 'target4']],
  axis = 1)

# Merge in date information
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  dates_with_info[['dailyDataDate', 'date', 'year', 'month', 'inSeason',
    'seasonPart']],
  on = ['dailyDataDate'],
  how = 'left'
  )

# Merge in some player information
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  players[['playerId', 'playerName', 'DOB', 'mlbDebutDate', 'birthCity',
    'birthStateProvince', 'birthCountry', 'primaryPositionName']],
   on = ['playerId'],
   how = 'left'
   )
# Merge in some player roster information by date
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  (rosters[['dailyDataDate', 'playerId', 'statusCode', 'status', 'teamId']].
    rename(columns = {
      'statusCode': 'rosterStatusCode',
      'status': 'rosterStatus',
      'teamId': 'rosterTeamId'
      })
    ),
  on = ['dailyDataDate', 'playerId'],
  how = 'left'
  )
# Merge in team name from player's roster team
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  (teams[['id', 'teamName']].
    rename(columns = {
      'id': 'rosterTeamId',
      'teamName': 'rosterTeamName'
      })
    ),
  on = ['rosterTeamId'],
  how = 'left'
  )
# Merge in some player game stats (previously aggregated) from that date
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  player_date_stats_agg,
  on = ['dailyDataDate', 'playerId'],
  how = 'left'
  )
# Merge in team name from player's game team
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  (teams[['id', 'teamName']].
    rename(columns = {
      'id': 'gameTeamId',
      'teamName': 'gameTeamName'
      })
    ),
  on = ['gameTeamId'],
  how = 'left'
  )
# Merge in some team game stats/results (previously aggregated) from that date
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  team_date_stats_agg.rename(columns = {'teamId': 'gameTeamId'}),
  on = ['dailyDataDate', 'gameTeamId'],
  how = 'left'
  )
# Merge in player transactions of note on that date
    
# Merge in some pieces of team standings (previously filter/processed) from that date
player_engagement_with_info = pd.merge(
  player_engagement_with_info,
  standings_for_digital_engagement_merge.
    rename(columns = {'teamId': 'gameTeamId'}),
  on = ['dailyDataDate', 'gameTeamId'],
  how = 'left'
  )
display(player_engagement_with_info)

In [ ]:
display(player_engagement_with_info.info(max_cols = 200))

In [ ]:
#### Functions to help with calculating & display grouped target summaries

def GetGroupedTargetValuesSummary(df_with_group_and_targets, 
  grouping_vars_list):
    
    # Group target values by specified variables
    group_target_values_summary = (df_with_group_and_targets.
      groupby(grouping_vars_list, as_index = False, dropna = False).
      agg(
        numPlayerDates = ('playerId', 'count'),
        numPlayers = ('playerId', 'nunique'),
        target1 = ('target1', np.mean),
        target2 = ('target2', np.mean),
        target3 = ('target3', np.mean),
        target4 =  ('target4', np.mean),
        target1To4Avg = ('targetAvg', np.mean)
        ).
      sort_values(['target1To4Avg'], ascending = False,
        ignore_index = True)
      )
    return(group_target_values_summary)
;
def CalcAndDisplayMultipleSetsOfGroupedTargetValues(
    df_with_group_and_targets, grouping_names_and_vars_sets_df):

    num_group_sets = grouping_names_and_vars_sets_df.shape[0]
    
    kaggle_data_tabs = widgets.Tab()

    # Add Output widgets for each group as tabs' children
    kaggle_data_tabs.children = list([widgets.Output() 
      for group_set in range(0, num_group_sets)])

    # Loop over each group (name/vars combo), create tab, calc/display summary
    for index, row in grouping_names_and_vars_sets_df.iterrows():
    
        this_group_name = row['groupName']
        this_group_vars_list = row['groupVarsList']
# Group target values by specified variables
        group_target_values_summary = GetGroupedTargetValuesSummary(
          df_with_group_and_targets, this_group_vars_list
          )

        # Rename tab bar titles to string concatenating grouping vars
        kaggle_data_tabs.set_title(index, this_group_name)

        # Display corresponding table output for this tab name
        with kaggle_data_tabs.children[index]:
            print('Average Daily Digital Engagement by ' + this_group_name)
            
            print('Average Target Values Grouped By ' + 
              str(this_group_vars_list))

            display(group_target_values_summary.round(decimals = 1))
    
    display(kaggle_data_tabs)
    ;

In [ ]:
player_related_grouping_names_and_var_sets = pd.DataFrame(data = {
  'groupName': [
    'Player',
    'Player Team (by Roster)',
    'Player Roster Status',
    
    ],
  'groupVarsList': pd.Series([
    ['playerId', 'playerName'], 
    ['rosterTeamName'],
    ['rosterStatusCode', 'rosterStatus'], 
    
    ])
 })    

CalcAndDisplayMultipleSetsOfGroupedTargetValues(
  player_engagement_with_info, 
  player_related_grouping_names_and_var_sets
  )

In [ ]:
# Players in the test set. We'll filter our data for only this set of players
pids_test = players.playerId.loc[
    players.playerForTestSetAndFuturePreds.fillna(False)
].astype(str)

# Name of target columns
targets = ["target1", "target2", "target3", "target4"]


def make_playerBoxScores(dfs: dict, features):
    X = dfs['playerBoxScores'].copy()
    X = X[['gameDate', 'playerId'] + features]
    # Set dtypes
    X = X.astype({name: np.float32 for name in features})
    X = X.astype({'playerId': str})
    # Create date index
    X = X.rename(columns={'gameDate': 'date'})
    X['date'] = pd.PeriodIndex(X.date, freq='D')
    # Aggregate multiple games per day by summing
    X = X.groupby(['date', 'playerId'], as_index=False).sum()
    return X

In [ ]:
 Y= player_engagement_with_info.iloc[:,[3,4,5,6,7]]  
 Y   

In [ ]:
X=player_engagement_with_info.iloc[:, np.r_[0:3,8:61]]
X.tail()
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_valid, y_train, y_valid=train_test_split(
        X,
        Y,
        test_size=30,
        shuffle=False,
    ) 
X_train.shape

In [ ]:
from torch.utils.data import DataLoader,Dataset
class MLBDataset(Dataset):
    def __init__(self, df, targets=None, mode='train'):
        self.mode = mode
        self.data = df
        if mode == 'train':
            self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            x = self.data[idx]
            y = np.array(self.targets[idx])
            return torch.from_numpy(x).float(), torch.from_numpy(y).float()
        elif self.mode == 'test':
            return torch.from_numpy(self.data[idx]).float()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt


device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#hyper parameters

input_size =140344176#flattening the array of size 28*28
hidden_size=1024
num_epochs = 100
batch_size = 32
val_btch_size=batch_size*8
learning_rate = 1e-2
targets = ['target1', 'target2', 'target3', 'target4']
#MNIST
train_dataset= MLBDataset(X_train,y_train)
val_dataset=MLBDataset(X_valid, y_valid)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle =True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=val_btch_size, shuffle=False)

'''examples = iter(train_loader)
features,targets= examples.next()
print(features.shape,targets.shape)
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(samples[i][0])
    #plt.show'''
class NeuralNet(nn.Module):
     def __init__(self,input_size,hidden_size):
         super(NeuralNet,self).__init__()
         self.l1= nn.Linear(input_size,hidden_size)
         self.relu = nn.ReLU()
         self.l2= nn.Linear(hidden_size,hidden_size)
         self.l3=nn.Linear(hidden_size,len(targets)) 
     def forward(self,x):
         out = self.l1(x)
         out = self.relu(out)
         out = self.l2(out)
         out = self.relu(l2)
         out =slef.l3(out)
         return out
model= NeuralNet(input_size,hidden_size)
#loss and optimizer
criterion= nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

#training loop
n_total_steps= len(train_loader)
for epoch in range(num_epochs):
    for i,(features,targets) in enumerate(train_loader):
        
        features= features.to(device)
        targets=labels.to(device)
        #forward pass
        outputs=model(features)
        loss=criterion(features,targets)
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100==0:
            print(f'epoch{epoch+1}/{num_epochs},step {i+1}/{n_total_steps},loss={loss.item():.4f}')
#Testing
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        outputs=model(images)
    
